In [2]:
import pandas as pd
from rdkit import Chem
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

paths = ["1-100", "101-200", "201-300", "301-400", "401-500", "501-600", "601-700", "701-800", "801-900", "901-1000"]

sample_smiles = []
for path in paths:
    df = pd.read_excel(f"sample/Substance_{path}.xlsx", header=4)
    x = df[df.columns[2]]
    x_iso = df[df.columns[3]]
    x = x.dropna().reset_index(drop=True)
    
    for i in range(len(x)):
        mol = Chem.MolFromSmiles(x[i])
    
        eject = "."
        if eject in x[i]:
            continue
    
        if type(x_iso[i]) == str:
            if "2H" in x_iso[i] or "3H" in x_iso[i]:
                continue
    
        carboxylic_acid = Chem.MolFromSmarts("C(=O)[OH]")
        num_acid = len(mol.GetSubstructMatches(carboxylic_acid))
        if num_acid >= 2:
            sample_smiles.append(x[i])


print(len(sample_smiles))


[16:38:54] WARNING: not removing hydrogen atom without neighbors
[16:38:54] Explicit valence for atom # 32 O, 3, is greater than permitted
[16:38:54] WARNING: not removing hydrogen atom without neighbors
[16:38:54] Explicit valence for atom # 12 O, 2, is greater than permitted
[16:38:54] WARNING: not removing hydrogen atom without neighbors
[16:38:54] Explicit valence for atom # 40 O, 3, is greater than permitted
[16:38:54] WARNING: not removing hydrogen atom without neighbors


707


In [17]:
from rdkit.Chem import AllChem

test = sample_smiles[453]

mol = Chem.MolFromSmiles(test)
mol = Chem.AddHs(mol)

AllChem.EmbedMolecule(mol)
AllChem.UFFOptimizeMolecule(mol)

conf = mol.GetConformer()
# atom_idx = 1
# conf.SetAtomPosition(atom_idx, (0.0, 0.0, 0.0))
for atom in mol.GetAtoms():
    pos = conf.GetAtomPosition(atom.GetIdx())
    print(f"{atom.GetSymbol()} {pos.x:.4f} {pos.y:.4f} {pos.z:.4f}")

O 3.6866 0.6568 -0.3783
C 3.1499 -0.4181 0.0025
O 3.9587 -1.4797 0.3999
C 1.6764 -0.5803 -0.0221
C 1.1591 -1.8722 -0.2669
C -0.2173 -2.0840 -0.3884
C -1.0791 -0.9990 -0.2650
C -0.6054 0.2650 -0.0160
C 0.7745 0.5152 0.1339
C 1.2338 1.8693 0.5187
O 2.1144 2.0118 1.4093
O 0.6741 2.9954 -0.0773
C -1.7236 1.2383 0.1173
C -2.9973 0.3833 -0.0049
C -2.5565 -1.0483 -0.3490
H 4.9679 -1.3913 0.4050
H 1.8246 -2.7179 -0.3900
H -0.6043 -3.0766 -0.5809
H 0.9718 3.9265 0.1881
H -1.6824 1.7487 1.1038
H -1.6797 1.9831 -0.7060
H -3.6649 0.7870 -0.7972
H -3.5490 0.3871 0.9608
H -2.8680 -1.3215 -1.3802
H -2.9643 -1.7786 0.3829


In [16]:
print(sample_smiles[453])

O=C(O)C1=CC=C2C(=C1C(=O)O)CCC2


In [5]:
mol = Chem.AddHs(Chem.MolFromSmiles("O=C(O)C1=CC=CC(C(=O)O)=C1C(=O)O"))
phthalic_pattern = Chem.MolFromSmarts("c1ccc(C(=O)O)c(C(=O)O)c1")

matches = mol.GetSubstructMatches(phthalic_pattern)
if matches:
    print("フタル酸構造を検出しました。")
    print(type(matches))
    for match in matches[0]:
        atom = mol.GetAtomWithIdx(match)
        symbol = atom.GetSymbol()
        print(f"Atom Index: {match}, Element: {symbol}")
        if symbol == "O":
            neighbors = atom.GetNeighbors()
            elements = [n.GetSymbol() for n in neighbors]
            if "H" in elements and "C" in elements:
                for n in neighbors:
                    if n.GetSymbol() == "H":
                        print(f"O: {atom.GetIdx()}, H: {n.GetIdx()}")
        elif symbol == "C":
            neighbors = atom.GetNeighbors()
            elements = [n.GetSymbol() for n in neighbors]
            if elements.count("O") >= 2:
                for n in neighbors:
                    print(f"C: {atom.GetIdx()}, locate: {n.GetIdx()}, Element: {n.GetSymbol()}")

else:
    print("フタル酸構造は存在しません。")

フタル酸構造を検出しました。
<class 'tuple'>
Atom Index: 4, Element: C
Atom Index: 5, Element: C
Atom Index: 6, Element: C
Atom Index: 7, Element: C
Atom Index: 8, Element: C
C: 8, locate: 7, Element: C
C: 8, locate: 9, Element: O
C: 8, locate: 10, Element: O
Atom Index: 9, Element: O
Atom Index: 10, Element: O
O: 10, H: 19
Atom Index: 11, Element: C
Atom Index: 12, Element: C
C: 12, locate: 11, Element: C
C: 12, locate: 13, Element: O
C: 12, locate: 14, Element: O
Atom Index: 13, Element: O
Atom Index: 14, Element: O
O: 14, H: 20
Atom Index: 3, Element: C


In [ ]:
test = sample_smiles[0]
test = "O=C(O)C1=CC=C2C(=C1C(=O)O)CCC2"

mol = Chem.MolFromSmiles(test)
mol = Chem.AddHs(mol)
phthalic_pattern = Chem.MolFromSmarts("c1ccc(C(=O)O)c(C(=O)O)c1")

AllChem.EmbedMolecule(mol)
AllChem.UFFOptimizeMolecule(mol)

matches = mol.GetSubstructMatches(phthalic_pattern)[0]

'''
conf = mol.GetConformer()
for atom in mol.GetAtoms():
    pos = conf.GetAtomPosition(atom.GetIdx())
    print(f"{atom.GetSymbol()} {pos.x:.4f} {pos.y:.4f} {pos.z:.4f}")
'''

oh_list = []
cooh_list = []

for match in matches:
    atom = mol.GetAtomWithIdx(match)
    symbol = atom.GetSymbol()
    print(f"Atom Index: {match}, Element: {symbol}")
    if symbol == "O":
        neighbors = atom.GetNeighbors()
        elements = [n.GetSymbol() for n in neighbors]
        if "H" in elements and "C" in elements:
            for neighbor in neighbors:
                if neighbor.GetSymbol() == "H":
                    oh_list.append((atom.GetIdx(), neighbor.GetIdx()))
for match in matches:
    atom = mol.GetAtomWithIdx(match)
    symbol = atom.GetSymbol()
    if symbol == "C":
        neighbors = atom.GetNeighbors()
        elements = [n.GetSymbol() for n in neighbors]
        if elements.count("O") >= 2:
            for neighbor in neighbors:
                if neighbor.GetSymbol() == "C":
                    c = neighbor.GetIdx()
                elif neighbor.GetSymbol() == "O":
                    if len(neighbor.GetNeighbors()) == 1:
                        o_double = neighbor.GetIdx()
                    elif len(neighbor.GetNeighbors()) == 2:
                        for oh in oh_list:
                            if oh[0] == neighbor.GetIdx():
                                o_single = oh[0]
                                h = oh[1]
            cooh_list.append((atom.GetIdx(), c, o_double, o_single, h))
            print(cooh_list)

0
Atom Index: 5, Element: C
Atom Index: 6, Element: C
Atom Index: 7, Element: C
Atom Index: 8, Element: C
Atom Index: 9, Element: C
Atom Index: 10, Element: O
Atom Index: 11, Element: O
Atom Index: 3, Element: C
Atom Index: 1, Element: C
Atom Index: 0, Element: O
Atom Index: 2, Element: O
Atom Index: 4, Element: C
[(9, 8, 10, 11, 18)]
[(9, 8, 10, 11, 18), (1, 3, 0, 2, 15)]


In [19]:
import numpy as np
from rdkit.Geometry import Point3D

df1 = pd.read_csv("sample/basis1.txt", sep="\\s+", names=["atom", "x", "y", "z"])
df2 = pd.read_csv("sample/basis2.txt", sep="\\s+", names=["atom", "x", "y", "z"])
df1 = df1[["x", "y", "z"]].to_numpy()
df2 = df2[["x", "y", "z"]].to_numpy()
df1_basis = [
    df1[0], df1[6], df1[1], 
    df1[10]-df1[6], df1[11]-df1[6], df1[17]-df1[6], 
    df1[21]-df1[6], df1[22]-df1[6], df1[23]-df1[6], 
    ]

df2_basis = [
    df2[1], df2[7], df2[0], 
    df2[9]-df2[7], df2[8]-df2[7], df2[16]-df2[7], 
    df2[18]-df2[7], df2[19]-df2[7], df2[20]-df2[7], 
    ]
df = [df1_basis, df2_basis]

water = [df2[18], df2[19], df2[20], df2[21], df2[22], df2[23]]


conf = mol.GetConformer()
coordinates = []
for atom in mol.GetAtoms():
    pos = conf.GetAtomPosition(atom.GetIdx())
    coordinates.append([pos.x, pos.y, pos.z])
    print(f"{atom.GetSymbol()} {pos.x:.4f} {pos.y:.4f} {pos.z:.4f}")

coordinates = np.array(coordinates)

c_cooh1 = cooh_list[0][0]
c_benzene1 = cooh_list[0][1]
c_cooh2 = cooh_list[1][0]
c_benzene2 = cooh_list[1][1]

r1 = [coordinates[c_benzene1],coordinates[c_benzene2]]
r2 = [coordinates[c_cooh1],coordinates[c_cooh2]]
r3 = [coordinates[c_benzene2], coordinates[c_benzene1]]
cooh_idx = [c_cooh1, c_cooh2]
benzene_idx = [c_benzene2, c_benzene1]

O 7.9343 0.0236 -0.0557
C 6.9441 -0.7534 0.0196
O 7.1468 -2.1246 -0.1119
C 5.5892 -0.2084 0.2902
C 5.4883 0.9923 1.0147
C 4.2450 1.5164 1.3571
C 3.0798 0.8585 0.9771
C 3.1393 -0.3277 0.2299
N 1.9156 -0.9674 -0.1744
C 0.8123 -0.2558 -0.7477
O 0.9446 0.9475 -1.1048
O -0.4048 -0.9203 -0.9309
C -1.5843 -0.2804 -1.4144
C -2.1893 0.6463 -0.3465
C -2.7093 -0.1230 0.8439
C -1.9786 -0.8682 1.7744
C -2.6659 -1.4897 2.8302
C -4.0621 -1.3495 2.9526
C -4.7818 -0.5811 2.0242
C -4.0789 0.0214 0.9803
C -4.5517 0.8950 -0.0713
C -5.8394 1.3639 -0.3337
C -6.0181 2.2442 -1.4128
C -4.9173 2.6422 -2.1967
C -3.6290 2.1607 -1.9097
C -3.4668 1.2764 -0.8398
C 4.4008 -0.8755 -0.1177
C 4.4686 -2.0638 -0.9953
O 3.8940 -3.1354 -0.6656
O 5.1856 -2.0035 -2.1868
H 8.0716 -2.5000 -0.2849
H 6.3781 1.5167 1.3416
H 4.1847 2.4331 1.9296
H 2.1236 1.2737 1.2699
H 1.8073 -1.9962 -0.0394
H -2.3219 -1.0647 -1.6915
H -1.3449 0.2963 -2.3344
H -1.4725 1.4314 -0.0244
H -0.9042 -0.9628 1.6915
H -2.1182 -2.0748 3.5574
H -4.5815 -1.82

In [20]:
for i in range(2):
    R = np.eye(3)
    r1 = [coordinates[c_benzene1],coordinates[c_benzene2]]
    t = -r1[i]
    coords = np.array([coordinates[i] + t for i in range(mol.GetNumAtoms())])
    
    # --- 4. 回転: C2をx軸上に ---
    x_vec = coords[cooh_idx[i]].copy()  # C2のベクトル
    x_axis = x_vec / np.linalg.norm(x_vec)
    
    # z軸との外積で回転軸を求める
    z = np.array([0, 0, 1])
    axis = np.cross(x_axis, np.array([1, 0, 0]))
    axis_norm = np.linalg.norm(axis)
    
    if axis_norm > 1e-8:
        axis /= axis_norm
        # 角度を求める
        theta = np.arccos(np.dot(x_axis, [1, 0, 0]))
        # 回転行列（ロドリゲスの回転公式）
        c, s = np.cos(theta), np.sin(theta)
        K = np.array(
            [[0, -axis[2], axis[1]], [axis[2], 0, -axis[0]], [-axis[1], axis[0], 0]]
        )
        R = np.eye(3) + s * K + (1 - c) * (K @ K)
        coords = coords @ R.T
    
    # --- 5. 回転: C3をxy平面上に（z成分を0にする） ---
    r3_new = coords[benzene_idx[i]].copy()
    # C3 の z 成分を 0 にするための回転軸は x 軸方向
    angle_z = np.arctan2(r3_new[2], r3_new[1])
    c, s = np.cos(-angle_z), np.sin(-angle_z)
    R2 = np.array([[1, 0, 0], [0, c, -s], [0, s, c]])
    coords = coords @ R2.T
    
    a = df[i][1].copy()
    a /= np.linalg.norm(a)
    b = coords[cooh_idx[i]].copy()
    b = b / np.linalg.norm(b)

    cos_theta1 = np.dot(a, b)
    if np.isclose(cos_theta1, 1.0, atol=1e-1):
        pass
    else:
        Ry = np.array([
            [-1, 0, 0],
            [0, -1, 0],
            [0, 0, 1]
        ])
        coords = coords @ Ry.T
    
    c = df[i][2].copy()
    c /= np.linalg.norm(c)
    d = coords[benzene_idx[i]]
    d = d / np.linalg.norm(d).copy()

    cos_theta2 = np.dot(c, d)
    if np.isclose(cos_theta2, 1.0, atol=1e-1):
        pass
    else:
        Rx = np.array([
            [1, 0, 0],
            [0, -1, 0],
            [0, 0, -1]
        ])
        coords = coords @ Rx.T
        print("x")
    
    dfi_copy = df[i].copy()
    coords[cooh_list[i][0]] = dfi_copy[1]
    dfi_copy += dfi_copy[1]


    for j in range(2,5):
        coords[cooh_list[i][j]] = dfi_copy[j+1]
    
    if i == 1:
        diff = df[1][2] - coords[cooh_list[0][1]]
        for j in cooh_list[0]:
            coords[j] += diff
    
    # --- 6. 新しい座標を反映 ---
    for j in range(mol.GetNumAtoms()):
        conf.SetAtomPosition(j, Point3D(*coords[j]))

    if i == 1:
        mol = Chem.RWMol(mol)
        for j in range(2):
            o = mol.AddAtom(Chem.Atom("O"))
            h1 = mol.AddAtom(Chem.Atom("H"))
            h2 = mol.AddAtom(Chem.Atom("H"))
            conf = mol.GetConformer()
            conf.SetAtomPosition(o, water[j*3])
            conf.SetAtomPosition(h1, water[j*3+1])
            conf.SetAtomPosition(h2, water[j*3+2])

    coordinates = coords

for atom in mol.GetAtoms():
    pos = conf.GetAtomPosition(atom.GetIdx())
    print(f"{atom.GetSymbol()} {pos.x:.4f} {pos.y:.4f} {pos.z:.4f}")

O 2.1523 -0.8218 -0.6072
C 1.4917 -0.0000 -0.0000
O 2.0163 0.9413 0.7628
C 0.0000 0.0000 0.0000
C -0.6647 -1.2381 -0.0440
C -2.0536 -1.2999 -0.1142
C -2.8058 -0.1300 -0.1335
C -2.1801 1.1240 -0.0635
N -2.9879 2.3145 -0.0540
C -4.1510 2.4651 0.7687
O -4.4068 1.6232 1.6735
O -4.9840 3.5717 0.5736
C -6.2066 3.7692 1.2812
C -7.2933 2.7971 0.7917
C -7.7022 3.0745 -0.6348
C -6.9318 2.9398 -1.7939
C -7.5231 3.2304 -3.0345
C -8.8705 3.6348 -3.1051
C -9.6398 3.7457 -1.9362
C -9.0294 3.4584 -0.7149
C -9.5897 3.4413 0.6187
C -10.8886 3.7173 1.0472
C -11.1779 3.5872 2.4149
C -10.1769 3.1801 3.3188
C -8.8774 2.9027 2.8627
C -8.5999 3.0468 1.5007
C -0.7622 1.1743 0.0000
C -0.1005 2.4937 0.1838
O -0.5317 3.3732 0.9019
O 0.9832 2.6511 -0.5699
H 3.0196 0.9301 0.7161
H -0.1051 -2.1656 -0.0505
H -2.5494 -2.2609 -0.1643
H -3.8838 -0.2005 -0.2071
H -2.7337 3.1102 -0.6790
H -6.5412 4.8172 1.1203
H -6.0323 3.6354 2.3711
H -6.9861 1.7365 0.9134
H -5.9019 2.6129 -1.7429
H -6.9413 3.1368 -3.9423
H -9.3179 3.851

In [23]:
from rdkit import Chem
from rdkit.Chem import Descriptors
test = "COc1ccc(NC(=O)NC23CC4CC(CC(C4)C2)C3)cc1"
mol = Chem.MolFromSmiles(test)
mol_wt = Descriptors.MolWt(mol)
ring = Descriptors.RingCount(mol)
logp = Descriptors.MolLogP(mol)
donor_num = Descriptors.NumHDonors(mol)
acceptor_num = Descriptors.NumHDonors(mol)
print("mol_wt:", mol_wt)
print("ring:", ring)
print("logp:", logp)
print("donor_num",donor_num)
print("acceptor_num",acceptor_num)

mol_wt = Descriptors.MolWt(mol)
if mol_wt >= 500:
    print("FALSE")

ring_num = Descriptors.RingCount(mol)
if ring_num > 4:
    print("FALSE")

logp = Descriptors.MolLogP(mol)
if logp < 1 or logp > 5:
    print("FALSE")

donor_num = Descriptors.NumHDonors(mol)
acceptor_num = Descriptors.NumHDonors(mol)
if donor_num + acceptor_num > 10:
    print("FALSE")

mol_wt: 300.40200000000004
ring: 5
logp: 3.7855000000000025
donor_num 2
acceptor_num 2
FALSE


In [18]:
n = 0
for i in range(5):
    if i == 2:
        print("skip")
        continue
    n += 1
    print(n)

1
2
skip
3
4
